In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
dbpia_aut = client['DBPIA']['Author']



In [ ]:
def filter(site, rawdata):
    if site == 'NTIS' :
        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""

    else :
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        
        if rawdata['paper_keyword'] == [] or rawdata['paper_keyword'] == 'None':
            keyword = []
        elif len(rawdata['paper_keyword']) > 1:
            for i in range(0, len(rawdata['paper_keyword'])):
                keyword = []
                keyword.append(rawdata['paper_keyword'][i].replace(" ", "").split("."))            
        else:
            keyword = rawdata['paper_keyword'].replace(" ", "").split(".")

        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']

    return coauthor, year, keyword, journal, conference, title

def Secondary_filter(name, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst = 0
    weight = 0
    joc = 0
    coauthor1, year1, keyword1, journal1, conference1, title1 = filter(site1, raw_one1)
    coauthor2, year2, keyword2, journal2, conference2, title2 = filter(site2, raw_one2)

    if inst1 == inst2:
        inst = 1
    else:
        inst = jaro.jaro_winkler_metric(inst1, inst2)

    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2 or inst >= 0.8: #or mng1 == mng2:
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else 0
    else:
        if inst >= 0.8:
            weight = 3
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    #print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [ ]:
raw_dbs = {'NTIS' : ntis_raw, 'SCIENCEON' : scion_raw, 'KCI' : kci_raw, 'DBPIA': dbpia_raw}
savetime1 = 0
savetime2 = 0
def getRaw(name):
    if 'raws' not in Answer_dict[name]:
        raws = []
        for site_one in site:
            if site_one in Answer_dict[name]:
                for c in raw_dbs[site_one].find({"_id": {"$in": Answer_dict[name][site_one]['papers']}}):
                    c['site'] = site_one
                    raws.append(c)
        
        Answer_dict[name]['raws'] = raws

processedList = []
deleteList = []

for Answer_one in Answer_dict :

    if '_' in Answer_one :
        start1 = time.time()
        name = Answer_one.split("_")
        if name[0] in processedList :
            continue
        preprocessedList = []
        c = 0
        while True :
            pname = name[0]+"_"+str(c)
            if pname in Answer_dict :            
                preprocessedList.append(pname)
                getRaw(pname)
                c += 1
            else :
                break
        end1 = time.time()
        savetime1 += end1 - start1
        processedList.append(name[0])
        flag = True
        while flag :
            flag = False
            pairs =list(itertools.combinations(preprocessedList, 2))
            
            for pair in pairs:
                pair = list(pair)

                raws1 = Answer_dict[pair[0]]['raws']
                raws2 = Answer_dict[pair[1]]['raws']
                
                for ra1, ra2 in zip(raws1, raws2):
                    site1 = ra1['site']
                    site2 = ra2['site']
                    inst1 = Answer_dict[pair[0]][site1]['oriInst']
                    inst2 = Answer_dict[pair[1]][site2]['oriInst']

                    if Secondary_filter(name[0], site1, inst1, ra1, site2, inst2, ra2) >= 3:
                        deleteList.append(pair[1])
                        for site_one in site:
                            if site_one in Answer_dict[pair[1]]:
                                if site_one in Answer_dict[pair[0]].keys() :                            
                                    Answer_dict[pair[0]][site_one]['A_id'].extend(Answer_dict[pair[1]][site_one]['A_id'])
                                    Answer_dict[pair[0]][site_one]['papers'].extend(Answer_dict[pair[1]][site_one]['papers'])
                                    Answer_dict[pair[0]]['raws'].extend(Answer_dict[pair[1]]['raws'])
                                else:
                                    Answer_dict[pair[0]][site_one] = Answer_dict[pair[1]][site_one]
                        flag = True
                        preprocessedList.remove(pair[1])
                        break
                if flag :
                    break
        end2 = time.time()
        savetime2 += end2 - end1    
for d in deleteList:
    del Answer_dict[d]

for d in Answer_dict : 
    if 'raws' in Answer_dict[d] :
        del Answer_dict[d]['raws']
# print(savetime1, savetime2)
print(Answer_dict)
# id_domestic.insert_many(Answer_dict.values()) #mongodb 추가